# Overview

This notebook shows how to implement a simple IR system using TF-IDF and includes several optional points for exploration.

# Important Note

Your homework will not look like this notebook. You'll be doing slightly lower-level programming that implements some of the features we see here.


In [2]:
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

import time

In [4]:
# Load in 10K different short biographies from the first paragraph of Wikipedia pages of people

with open('wiki-bios.small.txt', 'rt') as f:
    wiki_bios = f.readlines()
    
print(wiki_bios[0])

Career From June 2009 to February 2013, Sow was a technical advisor to the Prime Minister of Chad for microfinance and sustainable development. From February 2013 to October 2013, Sow was a technical advisor for Economic and Budgetary Affairs at the Presidency of the Republic. From October 2013 to April 2014, Sow held the post of Minister of Microcredits for the Promotion of Women and Youth. From April 2014 to August 2015, Sow was the Secretary of State for Finance and Budget in charge of microfinance. From November 2015 to August 2016, Sow was the Secretary General of the Court of Auditors. From August 2016 to February 2017, Sow was the Secretary of State for Infrastructure and Opening up. From February 5, 2017 to November 21, 2017, Sow was the Secretary of State for Finance and Budget. As of June 2018, Sow was the Chief of Staff to the President of Chad.



## Working with tf-idf

We'll be using the scikit-learn [implementation of tf-idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), which has a **lot of features**. For now let's use the default parameters.

In [27]:
# Create TfidfVectorizer object
vectorizer = TfidfVectorizer(sublinear_tf=True)

# Generate matrix of word vectors. Note that this fit_transform function will
# update the parameters of the vectorizer object so that it has a mapping from
# terms to which dimension they belong to
tfidf_matrix = vectorizer.fit_transform(wiki_bios)

# Print the shape of tfidf_matrix -- what do these dimensions represent?
print(tfidf_matrix.shape)

(10000, 93766)


In [15]:
# Let's take a peek at some of the term-index mapping 
for term, index in list(vectorizer.vocabulary_.items())[:10]:
    print('%s -> %d' % (term, index))

career -> 15578
from -> 32138
june -> 43857
2009 -> 1339
to -> 83975
february -> 30014
2013 -> 1351
sow -> 78305
was -> 89674
technical -> 82489


In [16]:
# How similar are documents? Let's try looking a small comparison

# Get a small matrix of just the first five docs
first5_docs = tfidf_matrix[:5]
print(first5_docs.shape)

# Compute and print the cosine similarity matrix
cosine_sim = cosine_similarity(first5_docs, first5_docs)
print(cosine_sim)

(5, 93766)
[[1.         0.08328871 0.00906274 0.09587704 0.11362392]
 [0.08328871 1.         0.0059176  0.07298004 0.10583887]
 [0.00906274 0.0059176  1.         0.00808131 0.03160891]
 [0.09587704 0.07298004 0.00808131 1.         0.12325786]
 [0.11362392 0.10583887 0.03160891 0.12325786 1.        ]]


In [17]:
# Record start time -- it's good to start thinking about time
start = time.time()

# Compute cosine similarity matrix -- what shape do you expect this to have?
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Print cosine similarity matrix
print(cosine_sim)

# Print time taken
print("Time taken: %s seconds" % (time.time() - start))

[[1.         0.08328871 0.00906274 ... 0.09534378 0.07690432 0.06348797]
 [0.08328871 1.         0.0059176  ... 0.10360765 0.07179313 0.07889376]
 [0.00906274 0.0059176  1.         ... 0.00334339 0.00226104 0.00238967]
 ...
 [0.09534378 0.10360765 0.00334339 ... 1.         0.09910909 0.10710182]
 [0.07690432 0.07179313 0.00226104 ... 0.09910909 1.         0.06714671]
 [0.06348797 0.07889376 0.00238967 ... 0.10710182 0.06714671 1.        ]]
Time taken: 5.328553199768066 seconds


In [18]:
# There are many ways to do something. In our case, scikit-learn's linear_kernel
# function is the same output but a different implementation. Perhaps it's faster?
# Let's find out!

# Record start time
start = time.time()

# Compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Print cosine similarity matrix
print(cosine_sim)

# Print time taken
print("Time taken: %s seconds" % (time.time() - start))

[[1.         0.08328871 0.00906274 ... 0.09534378 0.07690432 0.06348797]
 [0.08328871 1.         0.0059176  ... 0.10360765 0.07179313 0.07889376]
 [0.00906274 0.0059176  1.         ... 0.00334339 0.00226104 0.00238967]
 ...
 [0.09534378 0.10360765 0.00334339 ... 1.         0.09910909 0.10710182]
 [0.07690432 0.07179313 0.00226104 ... 0.09910909 1.         0.06714671]
 [0.06348797 0.07889376 0.00238967 ... 0.10710182 0.06714671 1.        ]]
Time taken: 5.4741740226745605 seconds


In [19]:
# How do we search? We need to get a query document in the same representation!
# We can use the vectorizer's transform function (not fit_tranform!) to turn
# the text into a vector

# Get the query vector
vectorized_query = vectorizer.transform(["queen born europe"])

# Compare they query vector with all the documents 
cosine_sim = linear_kernel(vectorized_query, tfidf_matrix)
cosine_sim.shape

(1, 10000)

In [20]:
# As a sanity check, let's take a look at the similarity. Here, we'll
# use enumerate so we get the document ID for which document has that
# similarity with the query.
doc_sims = list(enumerate(cosine_sim[0,:]))
doc_sims[:5]

[(0, 0.0),
 (1, 0.0072753530970886045),
 (2, 0.020192721216555482),
 (3, 0.004521908066136092),
 (4, 0.004509430162317532)]

In [28]:
# Let's wrap up all of these steps into a general "search" function
# that returns the 10 most-similar documents and their similarity
# so we can test it in class.

def search(query, vectorizer, tfidf_matrix, max_searched_docs=-1):

    # NOTE: we'll use this in an example in class so feel free to ignore
    # for now if just skimming the code
    if max_searched_docs > 0:
        tfidf_matrix = tfidf_matrix[:max_searched_docs,:]
    
    # Get the query vector
    vectorized_query = vectorizer.transform([query])

    # Get its similarties with all docs
    cosine_sim = linear_kernel(vectorized_query, tfidf_matrix)

    # Get a list of the doc IDs and simlarities 
    doc_sims = list(enumerate(cosine_sim[0,:]))

    # Sort the doc IDs based on the similarity scores
    doc_sims = sorted(doc_sims, key=lambda x: x[1], reverse=True)
    
    # Get the ids and scores for 10 most similar bios to the query
    top10_ids = doc_sims[:10]
    
    # Get the biography text of those doc ids
    top10_docs = [(wiki_bios[i], sim) for i, sim in top10_ids]
    
    return top10_docs

In [39]:
vectorizer = TfidfVectorizer(stop_words='english', min_df = 3)
tfidf_matrix = vectorizer.fit_transform(wiki_bios)
print(tfidf_matrix.shape)

(10000, 26183)


In [41]:
top10_docs = search("nba basketball player", vectorizer, tfidf_matrix)

# Print our "front page" of the search results
for rank, (doc, sim) in enumerate(top10_docs, 1):
    print('%d\t%f\t%s' % (rank, sim, doc))

1	0.493730	John Gilbert Wallace (born February 9, 1974) is a retired American professional basketball player. The 6'8 forward played seven seasons in the National Basketball Association (NBA), in addition to stints in Greece and Italy.

2	0.416402	Kennedy McIntosh (January 21, 1949 – March 6, 2009) was an American professional basketball player whose NBA career lasted from 1971-1975. At 6'7" tall, he played the forward position.

3	0.403508	William A. "Bill" Smith (born February 14, 1949) is a retired American professional basketball center who played two seasons in the National Basketball Association (NBA) as a member of the Portland Trail Blazers (1971–73). He was drafted by the Blazers in the third round (42 pick overall) of the 1971 NBA Draft from Syracuse University, where he holds the individual player single game scoring record of 47 points achieved on January 14, 1971 against Lafayette University.

4	0.357325	Tommy Núñez is the founder of the Tommy Núñez foundation and a former

## In-class question 1: What kinds of queries work well or badly?

Queries that specifies a detailed name of a person. 

## In-class question 2: What happens if we adjust the TfIdfVectorizer's parameters?

## In-class question 3: How might we evaluate whether one version is better than another?

## In-class question 4: How well is this approach going to scale?